In [35]:
import pandas as pd

df = pd.read_csv('vendas_modificado.csv') # Isso lê o csv
df.head() # Exibe as primeiras linhas

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [36]:
print(df['hora'].unique()) # Mostra os valores unicos na coluna hora, bom pra ver os que estão diferentes

['23:35:51' '09:00:53' '15:30:28' ... '02:45:20' '13:18:40' '08:14:34']


In [37]:
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.time # Muda a hora pro formato certo
df['data'] = pd.to_datetime(df['data'], errors='coerce') # Myda a data pro formato certo
df[['data', 'hora']].head() # Exibe só a data e a hora

,data,hora
0,2021-03-20,23:35:51
1,2020-10-30,09:00:53
2,2021-06-09,15:30:28
3,2022-06-04,08:41:23
4,2019-05-04,13:38:45


In [ ]:
# Removendo símbolos de R$ e transformando colunas em float

colunas_monetarias = ['valor', 'frete', 'total']

for col in colunas_monetarias:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r'R\$\s?', '', regex=True) # remove R$ e espaços
        .str.replace(',', '.', regex=False)    # troca vírgula por ponto
    )
    # Tenta converter para float, se não conseguir coloca NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convertendo quantidade
df['quantidade'] = pd.to_numeric(df['quantidade'], errors='coerce')
df['quantidade'] = df['quantidade'].fillna(1)

# Mostra as colunas
df[['valor', 'quantidade', 'frete', 'total']].head()


,valor,quantidade,frete,total
0,16.87,13,20.0,239.31
1,3.25,3,0.0,9.75
2,1.63,9,22.0,36.67
3,8.74,4,20.0,54.96
4,9.48,2,0.0,18.96


In [ ]:
# Recalculando o total com base na fórmula: total = valor * quantidade + frete
df['total_calculado'] = df['valor'] * df['quantidade'] + df['frete']
df['diferenca_total'] = df['total'] - df['total_calculado']

# Mostrando com diferença no total
df[df['diferenca_total'].abs() > 0.01][['id_da_compra', 'valor', 'quantidade', 'frete', 'total', 'total_calculado']]


,id_da_compra,valor,quantidade,frete,total,total_calculado
64,56909,586.88,6,0.0,26.88,3521.28
71,17370,16.51,14,-10.0,241.14,221.14
73,13612,2.90,13,110.0,48.70,147.70
74,39475,3.94,10,5992.5,47.90,6031.90
81,45815,2.63,2,7755.0,16.26,7760.26
...,...,...,...,...,...,...
368712,30905,11.25,7,5992.5,87.25,6071.25
368726,4709,3.21,10,170.0,49.10,202.10
368731,47461,989.05,1,0.0,7.55,989.05
368735,40352,2022.64,11,10.5,180.34,22259.54


In [48]:
# Substituindo o total antigo pelo recalculado
df['total'] = df['total_calculado']

# Removendo colunas auxiliares
df.drop(columns=['total_calculado', 'diferenca_total'], inplace=True)


In [49]:
df.drop_duplicates(inplace=True) # Remove as duplicatas


In [50]:
df.head() # teste pra ver se deu tudo certo

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [51]:
df.to_csv('dados_limpos.csv', index=False) # Exporta um csv limpo
print("Deu certo")


Deu certo


In [ ]:
df.info() #informações


<class 'pandas.core.frame.DataFrame'>
Index: 363594 entries, 0 to 368751
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id_da_compra  363594 non-null  int64         
 1   data          363594 non-null  datetime64[ns]
 2   hora          363594 non-null  object        
 3   cliente       363594 non-null  object        
 4   produto       363594 non-null  object        
 5   valor         363511 non-null  float64       
 6   quantidade    363594 non-null  int64         
 7   total         356242 non-null  float64       
 8   status        363594 non-null  object        
 9   cidade        363594 non-null  object        
 10  estado        363594 non-null  object        
 11  pais          363594 non-null  object        
 12  cep           363594 non-null  object        
 13  frete         356324 non-null  float64       
 14  pagamento     363594 non-null  object        
 15  vendedor      359963 n

Relatório:

Etapas da Limpeza Realizadas

1. Verificação e Padronização de Tipos e Formatos

- Datas e Horas:
  - Conversão da coluna `data` para o formato `YYYY-MM-DD`
  - Conversão da coluna `hora` para o formato `HHMMSS`
- Valores Numéricos:
  - Colunas `valor`, `frete` e `total` foram convertidas de strings com símbolo "R$" para `float`
  - Coluna `quantidade` convertida para tipo numérico

---

2. Tratamento de Inconsistências

- Valores Ausentes:
  - Valores nulos na coluna `quantidade` foram preenchidos com `1`
- Registros Duplicados:
  - Remoção de duplicatas com base na coluna `id_da_compra` (se necessário)
- Verificação de Cálculos:
  - Verificação se `total ≈ valor * quantidade + frete`
  - Correção de registros com cálculos incorretos (se aplicável)

---

Daqui pra baixo são só alguns testes pra explorar o csv:

In [ ]:
print('Compra mais cara:')
df.loc[df['total'].idxmax()]



Compra mais cara:


id_da_compra                        5444
data                 2022-04-01 00:00:00
hora                            15:12:05
cliente               TIAGO VICTOR ALVES
produto                     Refrigerante
valor                               6.54
quantidade                            10
total                           162215.4
status                      Em Separação
cidade                    Angra dos Reis
estado                                RJ
pais                              Brasil
cep                            23900-000
frete                           162150.0
pagamento              Cartão de Crédito
vendedor        CARLOS QUEIROZ DE AGUIAR
marca                          Coca-Cola
Name: 341042, dtype: object

In [61]:
print('Top 10 produtos mais vendidos em quantidade:')
df.groupby('produto')['quantidade'].sum().sort_values(ascending=False).head(10)

Top 10 produtos mais vendidos em quantidade:


produto
Pasta de Dente      202797
Queijo Mussarela    202036
Sabonete            192113
Manteiga            188312
Papel Toalha        145506
Desinfetante        142758
Cerveja             118671
Molho de Tomate      96853
Condicionador        88811
Macarrão             74158
Name: quantidade, dtype: int64

In [ ]:
print ('Valor total de vendas por estado:')
df.groupby('estado')['total'].sum().sort_values(ascending=False)


estado
MG           45306371.73
RJ           23618962.91
SP            8569921.87
São Paulo      133217.65
MTSa           118920.11
RS              22431.20
PR              19896.49
PSC              7393.22
Name: total, dtype: float64

In [63]:
print ('Produtos com frete mais caro:')
df.groupby('produto')['frete'].mean().sort_values(ascending=False).head(10)


Produtos com frete mais caro:


produto
Biscoito Recheado#$@!    149.075758
Arroz#$@!                125.291971
Presunto#$@!             106.870732
Refrigerante#$@!          88.583744
Condicionador#$@!         84.526860
Sal                       82.518896
Suco de Laranja#$@!       81.349727
Shampoo#$@!               80.364943
Sabão em Pó               74.949578
Refrigerante              74.260996
Name: frete, dtype: float64